In [1]:
import os
import pandas as pd
import sys

from pathlib import Path

# Custom imports
sys.path.append(os.path.join(
    os.path.abspath('.'), 'notebooks'))

from commons import *

In [2]:
DIR_PER = DIR_processed / 'PER'

FILE_ids2players = 'Ids_to_Players.csv'
FILE_ids2teams = 'Ids_to_Teams.csv'

In [3]:
def factor(row_ptl: pd.DataFrame) -> float:
    return 2/3 - (0.5*row_ptl['Ast'] / row_ptl['FGM']) \
               / (2.0*row_ptl['FGM'] / row_ptl['FTM'])

In [4]:
def vop(row_ptl: pd.DataFrame) -> float:
    """
    Value of possession.
    """
    den = row_ptl['FGA'] - row_ptl['OReb'] \
        + row_ptl['TO'] + 0.44*row_ptl['FTA']
    
    return row_ptl['Pts'] / den

In [5]:
def drbp(row_ptl: pd.DataFrame) -> float:
    """
    Defensive rebounds percentage.
    """
    return row_ptl['DReb'] / row_ptl['Reb']

In [6]:
def uper(row_ptl: pd.DataFrame) -> float:
    """
    Unadjusted PER.
    """
    a = row_ptl['3PM_p'] \
      - (row_ptl['PF_p']*row_ptl['FTM']) / row_ptl['PF']
    
    b = row_ptl['FTM_p']/2
    b = b*(2 - row_ptl['Ast_t'] / (3*(row_ptl['FGM_t'] + 1e-1)))
    
    c = row_ptl['FGM_p'] * (2 - factor(row_ptl)*row_ptl['Ast_t'] \
                                               / (row_ptl['FGM_t'] + 1e-1))
    
    d = 2*row_ptl['Ast_p'] / 3
    
    e = drbp(row_ptl)*(2*row_ptl['OReb_p']
                           + row_ptl['Blk_p'] 
                           - 0.2464*(row_ptl['FTA_p'] - row_ptl['FTM_p'])
                           - (row_ptl['FGA_p'] - row_ptl['FGM_p'])
                           - row_ptl['Reb_p'])                \
      + 0.44*row_ptl['FTA']*(row_ptl['PF_p'] / row_ptl['PF']) \
      - (row_ptl['TO_p'] + row_ptl['OReb_p'])                 \
      + row_ptl['Stl_p'] + row_ptl['Reb_p']                   \
      - 0.1936*(row_ptl['FTA_p'] - row_ptl['FTM_p'])
    
    f = vop(row_ptl)*e

    return (1/(row_ptl['Min_p'] + 1e-1))*(a + b + c + d + f)

In [7]:
def per(row_ptl: pd.DataFrame,
        uper_mean: float) -> float:
    """
    Player efficiency rating (PER) by J. Hollinger.  
    Ref. https://en.wikipedia.org/wiki/Player_efficiency_rating
    """ 
    _per = (row_ptl['uPER']*row_ptl['Pace']) / row_ptl['Pace_t']
    
    return _per*15/uper_mean

In [8]:
per_1996_20 = []

df_ids2players = pd.read_csv(DIR_processed / FILE_ids2players)
df_ids2teams   = pd.read_csv(DIR_processed / FILE_ids2teams)

for y in range(YEAR_start, YEAR_end + 1):
    filename = year2filename(y)
    
    # Load all data for year y
    df_pace    = pd.read_csv(DIR_raw_pace / filename)
    df_players = pd.read_csv(DIR_raw_players / filename)
    df_teams   = pd.read_csv(DIR_raw_teams / filename)
    df_league  = pd.read_csv(DIR_processed_league / filename)
    
    # Merge players DF with Ids info
    df_players['Player'] = df_players['Player'] \
        .apply(lambda s: unicd2ascii(s))

    df_players = pd.merge(df_players, df_ids2teams,
                          left_on='Team', right_on='Id') \
                   .rename(columns={'Name': 'Team_full'})                          \
                   .drop('Id', 1) 

    df_players = pd.merge(df_players, df_ids2players,
                          left_on='Player', right_on='Name') \
                   .drop(['Age', 'Name'], 1)
    
    # Merge teams DF with pace info
    df_teams   = pd.merge(df_teams, df_pace, on='Team')
    
    
    # Join players and teams DFs
    df_pt = pd.merge(df_players, df_teams,
                     left_on='Team_full', right_on='Team',
                     suffixes=('_p', '_t'))
    df_pt = df_pt.drop('Team_t', 1) \
                 .rename(columns={'Team_p': 'Team',
                                  'Pace': 'Pace_t'})

    df_nl = pd.concat([df_league]*len(df_pt), ignore_index=True)

    # Join pt and league DFs
    df_ptl = pd.concat([df_pt, df_nl], axis=1) \
               .set_index('Id')
    
    # Compute the unadjusted PER
    df_ptl['uPER'] = df_ptl.apply(uper, axis = 1)

    uper_mean = df_ptl['uPER'].mean()
    
    # Compute the PER adjusted with league's uPER
    df_ptl['PER'] = df_ptl.apply(lambda r: per(r, uper_mean), axis = 1)

    df_ptl.sort_values(by='PER', ascending=False,
                       inplace=True)
    
    sub_df_ptl = df_ptl[['Player', 'GP_p', 'PER']] \
                 .rename(columns={'GP_p': 'GP'})
    
    sub_df_ptl['Season'] = year2rangestr(y)
    
    # Collect all PER-rich sub DFs
    per_1996_20.append(sub_df_ptl)

In [9]:
df_per_1996_20 = pd.concat(per_1996_20).sort_index()

# Group by NBA player Id
groupby_per_1996_20 = df_per_1996_20.groupby(df_per_1996_20.index)

df_per_1996_20['PER_avg'] = groupby_per_1996_20['PER'].mean()
df_per_1996_20['GP_avg']  = groupby_per_1996_20['GP'].mean()

In [10]:
df_seasons_1996_20 = groupby_per_1996_20.size() \
    .reset_index(name='Seasons')                \
    .set_index('Id')

In [11]:
df_avg_per_1996_20 = df_per_1996_20[['Player', 'PER_avg', 'GP_avg']].drop_duplicates()

# Filter out outliers who have played only
# 1 clutch game each NBA playoffs on average
df_avg_per_1996_20 = df_avg_per_1996_20[df_avg_per_1996_20['GP_avg'] > 3.0]

df_avg_per_1996_20 = df_avg_per_1996_20.sort_values(by=['PER_avg'], ascending=False)
df_avg_per_1996_20 = df_avg_per_1996_20.join(df_seasons_1996_20)

df_avg_per_1996_20.head(50)

,Player,PER_avg,GP_avg,Seasons
Id,,,,
Jordami01,Michael Jordan,944.045109,11.500000,2
Drexlcl01,Clyde Drexler,795.429145,4.500000,2
Besttr01,Travis Best,563.140198,4.250000,4
Longllu01,Luc Longley,502.371956,7.000000,2
Hawkihe01,Hersey Hawkins,354.805181,4.500000,2
Kempsh01,Shawn Kemp,339.059437,3.333333,3
Roseja01,Jalen Rose,334.010714,6.500000,4
Olajuha01,Hakeem Olajuwon,315.568460,3.333333,3
Kerrst01,Steve Kerr,293.182843,4.600000,5


In [12]:
df_avg_per_1996_20.to_csv(DIR_PER / '1996-20.csv')